In [25]:
import os

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
from langchain.prompts import PromptTemplate

os.environ["OPENAI_API_KEY"] = "random-string"

In [26]:
doc1 = Document(page_content="Prem is an easy to use open source AI platform. With Prem you can quickly build provacy preserving AI applications.")
doc2 = Document(page_content="""
Prem App

An intuitive desktop application designed to effortlessly deploy and self-host Open-Source AI models without exposing sensitive data to third-party.

""")
doc3 = Document(page_content="""
Prem Benefits

Effortless Integration
Seamlessly implement machine learning models with the user-friendly interface of OpenAI's API.

Ready for the Real World
Bypass the complexities of inference optimizations. Prem's got you covered.

Rapid Iterations, Instant Results
Develop, test, and deploy your models in just minutes.

Privacy Above All
Your keys, your models. We ensure end-to-end encryption.

Comprehensive Documentation
Dive into our rich resources and learn how to make the most of Prem.

Preserve Your Anonymity
Make payments with Bitcoin and Cryptocurrency. It's a permissionless infrastructure, designed for you.
""")

In [27]:
# Using vicuna-7b-q4
chat = ChatOpenAI(openai_api_base="http://localhost:8001/api/v1", max_tokens=128)

# Using sentence transformers all-MiniLM-L6-v2
embeddings = OpenAIEmbeddings(openai_api_base="http://localhost:8000/api/v1")

# Using locally running Qdrant
url = "http://localhost:6333"

vectorstore = Qdrant.from_documents(
    [doc1, doc2, doc3], 
    embeddings, 
    url=url, 
    collection_name="prem_collection_test",
)

query = "What are Prem Benefits?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)


Prem Benefits

Effortless Integration
Seamlessly implement machine learning models with the user-friendly interface of OpenAI's API.

Ready for the Real World
Bypass the complexities of inference optimizations. Prem's got you covered.

Rapid Iterations, Instant Results
Develop, test, and deploy your models in just minutes.

Privacy Above All
Your keys, your models. We ensure end-to-end encryption.

Comprehensive Documentation
Dive into our rich resources and learn how to make the most of Prem.

Preserve Your Anonymity
Make payments with Bitcoin and Cryptocurrency. It's a permissionless infrastructure, designed for you.



In [28]:
template = """
You are an AI assistant for answering questions about Prem.
Provide a conversational answer to the question based on the following docouments found using semantic search. Be original, concice, accurate and helpful.

Question: {question}
=========
Context: {context}
=========
Answer in Markdown:
"""  # noqa E501
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=template,
)
chain = LLMChain(llm=chat, prompt=prompt, verbose=True)

question = "What are Prem Benefits?"
docs = vectorstore.similarity_search(question)
context = docs[0].page_content
chain.run(question=question, context=context)



> Entering new LLMChain chain...
Prompt after formatting:

You are an AI assistant for answering questions about Prem.
Provide a conversational answer to the question based on the following docouments found using semantic search. Be original, concice, accurate and helpful.

Question: What are Prem Benefits?
Context: 
Prem Benefits

Effortless Integration
Seamlessly implement machine learning models with the user-friendly interface of OpenAI's API.

Ready for the Real World
Bypass the complexities of inference optimizations. Prem's got you covered.

Rapid Iterations, Instant Results
Develop, test, and deploy your models in just minutes.

Privacy Above All
Your keys, your models. We ensure end-to-end encryption.

Comprehensive Documentation
Dive into our rich resources and learn how to make the most of Prem.

Preserve Your Anonymity
Make payments with Bitcoin and Cryptocurrency. It's a permissionless infrastructure, designed for you.

Answer in Markdown:


> Finished chain.


'\nPrem is an API that makes it easy to integrate machine learning models into your projects. With its user-friendly interface, you can seamlessly incorporate state-of-the-art models from OpenAI, without having to deal with the complexities of inference optimizations. Additionally, Prem enables rapid iterations and instant results, so you can develop, test, and deploy your models in just minutes.\n\nPrivacy is a top priority for Prem, as your keys and models are fully encrypted. The platform also provides comprehensive documentation to help you make the most of its capabilities. You can preserve your anonym'